#### Week 1: Vector Search Applications w/ LLMs.  Authored by Chris Sanchez.

# Project 1.3

# Overview
***
Welcome to the final notebook for Week 1! Take a look at all the ground we've covered so far:
- Chunking/splitting
- Vectorization of text
- Combining with metadata
- Saving to disk
- Class Configuration
- Data Indexing
- Keyword search
- Vector search
- OPTIONAL: Searching with Filters

We are now prepared to move on to a very important topic, **Retrieval Evaluation**.  I hope you've noticed that the search results will differ (sometimes slightly, sometimes by a lot) depeding on which search method you used: `keyword_search` or `vector_search`.  As humans, it's fairly easy for us to determine whether the returned search results are relevant to the query that was submitted, (though even here there will be differing opinions on result relevance).  But how do we systematically determine which search method is better in general?  And how do we measure the relative performance of our retrieval system if we change one of it's parameters...for example, changing our embedding model? What about measuring system performance over time as more documents are added to our datastore?

We need a way to evaluate our retrieval system, and this notebook will show you "one way" of doing that.  I say "one way" because there are many ways to approach this problem, and the method I'm showing you is not perfect (if anything it's a bit too conservative).  Ultimately, measuring retrieval performance is hard because it requires a lot of time and effort, and absent any user [click-data](https://en.wikipedia.org/wiki/Click_tracking), requires some form of data labeling.  With the advent of powerful generative LLMs the process of measuring retrieval performance has become much easier. Let's take a look how that works.

# Retrieval Evaluation - Process
***
Here's a high-level overview of how this Retrieval Evaluation process works:

1. Generate a "golden dataset" of query-context pairs, from a random selection of documents from the Impact Theory corpus.  I used a pseudo-LlamaIndex implementation for this step.
   - **Assumptions**:
     - The generated query-context pairs are, in fact, relevant to one another i.e. the query can be answered by the context that it's paired with
     - The generated queries are simliar in style and length to the type of queries that end users would ask
2. The golden dataset consists of three primary keys: `corpus`, `relevant_docs`, and `queries`
     - The `corpus` is the original text context/chunk with it's associated `doc_id`
     - The `queries` are the LLM generated queries, one (or more) for each entry in the `corpus`
     - The `relevant_docs` is a simple lookup table linking the `corpus` docs to the generated `queries`
3. We pass the golden dataset into a retrieval evluation function which does the following:
   - Takes in a `retriever` arg (`WeaviateClient`) and a few other configuration params
   - Iterates over all queries in the golden dataset and retrieves search results for each query from Weaviate datastore
   - Extracts all `doc_id` values from the retrieved results
   - Extracts the `doc_id` from the associated `relevant_docs` for each query
   - Checks if the relevant doc_id is in the list of retrieved result doc_ids
   - After all queries are completed a `hit_rate` score and `mrr` score are calculated for the entire golden dataset
   - Writes results to an `eval_results` folder

#### In a Nutshell
Ulitmately, given a golden dataset consisting of queries, relevant docs, and their associated doc_ids, the `retrieval_evaluation` function is checking if the relevant doc_id is found in the list of retrieved results doc_ids, for each query.

#### Problems with this Approach
The problems with this approach are many, I'll cover a few here:
- The **Assumptions** (see section 1 above) about the golden dataset must hold true.  Given that the pairs are AI generated (I used `gpt-3.5-turbo`), I think the first assumption will generally be true.  When reviewing the dataset I did find a few questions that were not answerable given the context, but for the most part they were.  The 2nd assumption though, is going to be dependent on your particular search use case.  I think for the purposes of this course, the questions generated are a decent reflection of how someone would query this dataset, and therefore do the job of measuring retriever performance.  But I would always check a real-world query distribution before using an approach like the one presented here.
- This approach is conversative in that there is only "one" right answer.  Either the relevant `doc_id` is in the results list or it isn't.  In reality, there are going to be several documents that could potentially answer the generated query, but we have no way to account for these other relevant documents, unless of course, we want to manually add doc_ids to the golden dataset (and depending on your business case, you may actually want to do that).
- We aren't measuring recall or precision because we aren't classifying other documents as "negatives".  As was just mentioned, the other documents in the results list may or may not be good matches, we just don't know.  Because we don't know, we can't really classify the other documents as "negatives".  So for this approach, we are measuring the ["hit rate"](https://uplimit.com/course/vector-search-apps/admin2/content/session_cln9hzpkl00721aah4hbz06fc/module/module_clo3hmyh0006p12cb3bmygky4) which is simply a count of the number of times that we found a relevant `doc_id` match in the results list and [Mean Reciprocal Rank (MRR)](https://uplimit.com/course/vector-search-apps/admin2/content/session_cln9hzpkl00721aah4hbz06fc/module/module_clo3hmyh0006p12cb3bmygky4).  We're using MRR over other metrics such as Mean Average Precision (MAP) because we are only looking at a [single relevant answer](https://stats.stackexchange.com/questions/127041/mean-average-precision-vs-mean-reciprocal-rank).  Hit rate is a good enough metric for determining if our retriever is retrieving quality results, and MRR will become more important later on when we add a Reranker to the mix.  

# Assignment 1.3
***
#### Instructions:
* Import the golden dataset as using the LlamaIndex `EmbeddingQAFinetuneDataset` Class
  - **side note: The `EmbeddingQAFinetuneDataset` Class is the same class used for creating fine-tuning datasets

In [99]:
%load_ext autoreload
%autoreload 2

#external libraries
from llama_index.finetuning import EmbeddingQAFinetuneDataset

#standard library imports
from typing import List, Tuple 
import time
import os

# utilities
from tqdm.notebook import tqdm
from rich import print
from dotenv import load_dotenv
env = load_dotenv('./.env', override=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load QA dataset

In [40]:
golden_dataset = EmbeddingQAFinetuneDataset.from_json("./data/golden_dataset.json")
len(golden_dataset.queries)
# list(golden_dataset.queries.values())
# golden_dataset.corpus

250

### Instantiate Weaviate client and set Class name

In [94]:
client = WeaviateClient(os.environ['WEAVIATE_API_KEY'], os.environ['WEAVIATE_ENDPOINT'])
class_name = 'Final_test'

#check if WCS instance is live and ready
client.is_live(), client.is_ready()

(True, True)

In [18]:
resp = client.hybrid_search('Why do some people perceive Jordan Peterson as being mean, even though he believes he is trying to help them see the reality of hierarchies and the need for personal growth?', class_name, limit=5)

### Evaluate Retrieval methods

In [104]:
from weaviate_interface import WeaviateClient
from retrieval_evaluation import calc_hit_rate_scores, calc_mrr_scores, record_results

def retrieval_evaluation(dataset: EmbeddingQAFinetuneDataset, 
                         class_name: str, 
                         retriever: WeaviateClient,
                         retrieve_limit: int=5,
                         chunk_size: int=256,
                         display_properties: List[str]=['doc_id', 'content']
                         ) -> Tuple[int, int, int]:

    results_dict = {'n':retrieve_limit, 
                    'Retriever': retriever.model_name_or_path, 
                    'chunk_size': chunk_size,
                    'kw_hit_rate': 0,
                    'kw_mrr': 0,
                    'vector_hit_rate': 0,
                    'vector_mrr': 0,
                    'total_misses': 0,
                    'total_questions':0
                    }
        
    start = time.perf_counter()
    for query_id, q in tqdm(dataset.queries.items(), 'Queries'):
        results_dict['total_questions'] += 1
        
        #make Keyword, Vector, and Hybrid calls to Weaviate host
        try:
            kw_response = retriever.keyword_search(request=q, class_name=class_name, limit=retrieve_limit, display_properties=display_properties)
            vector_response = retriever.vector_search(request=q, class_name=class_name, limit=retrieve_limit, display_properties=display_properties)
            
            #collect doc_ids to check for document matches (include only results_top_k)
            kw_doc_ids = {result['doc_id']:i for i, result in enumerate(kw_response, 1)}
            vector_doc_ids = {result['doc_id']:i for i, result in enumerate(vector_response, 1)}
            
            #extract doc_id for scoring purposes
            doc_id = dataset.relevant_docs[query_id][0]
     
            #increment hit_rate counters and mrr scores
            if doc_id in kw_doc_ids:
                results_dict['kw_hit_rate'] += 1
                results_dict['kw_mrr'] += 1/kw_doc_ids[doc_id]
            if doc_id in vector_doc_ids:
                results_dict['vector_hit_rate'] += 1
                results_dict['vector_mrr'] += 1/vector_doc_ids[doc_id]

            # if no hits, let's capture that
            else:
                # print(kw_response)
                results_dict['total_misses'] += 1
                
        except Exception as e:
            print(e)
            continue
    print(results_dict['kw_hit_rate'])
    print(results_dict['vector_hit_rate'])
    
    #use raw counts to calculate final scores
    calc_hit_rate_scores(results_dict)
    calc_mrr_scores(results_dict)
    
    end = time.perf_counter() - start
    print(f'Total Processing Time: {round(end/60, 2)} minutes')
    record_results(results_dict, chunk_size, './eval_results', as_text=True)
    return results_dict

### Run evaluation over golden dataset

In [101]:
golden_100 = EmbeddingQAFinetuneDataset.from_json('./data/golden_100.json')

In [105]:
retrieval_evaluation(golden_100, class_name, client, retrieve_limit=5)

Queries:   0%|          | 0/100 [00:00<?, ?it/s]

72

37

Total Processing Time: 0.39 minutes

{'n': 5,
 'Retriever': 'sentence-transformers/all-MiniLM-L6-v2',
 'chunk_size': 256,
 'kw_hit_rate': 0.72,
 'kw_mrr': 0.6,
 'vector_hit_rate': 0.37,
 'vector_mrr': 0.29,
 'total_misses': 63,
 'total_questions': 100}

In [90]:
results = {'n': 10,
 'top_k': 5,
 'alpha': 0.3,
 'Retriever': 'sentence-transformers/all-MiniLM-L6-v2',
 'Ranker': 'None',
 'chunk_size': 256,
 'kw_hit_rate': 0.68,
 'kw_mrr': 0.55,
 'vector_hit_rate': 0.4,
 'vector_mrr': 0.29,
 'hybrid_hit_rate': 0.71,
 'hybrid_mrr': 0.56,
 'total_misses': 73,
 'total_questions': 250}